In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import scipy
import numpy as np

In [2]:
data = pd.read_csv("./data.csv")
ports = data["Source Port"].unique()
ports = ports[ports !=1883]
data


,No.,Time,Source,Destination,Protocol,Length,Topic,Message Identifier,Source Port,Destination Port,Info
0,641,21.266512,192.168.15.4,192.168.15.3,MQTT,90,NaN,NaN,36061,1883,Connect Command
1,642,21.268032,192.168.15.4,192.168.15.3,MQTT,90,NaN,NaN,60501,1883,Connect Command
2,643,21.268601,192.168.15.4,192.168.15.3,MQTT,90,NaN,NaN,59967,1883,Connect Command
3,645,21.269093,192.168.15.4,192.168.15.3,MQTT,90,NaN,NaN,37215,1883,Connect Command
4,646,21.269524,192.168.15.3,192.168.15.4,MQTT,70,NaN,NaN,1883,36061,Connect Ack
...,...,...,...,...,...,...,...,...,...,...,...
1055,3177,124.997037,192.168.15.3,192.168.15.4,MQTT,70,NaN,104.0,1883,59967,Publish Ack (id=104)
1056,3179,124.998629,192.168.15.4,192.168.15.3,MQTT,97,Deviceb'Device - 4',104.0,60501,1883,Publish Message (id=104) [Deviceb'Device - 4']
1057,3180,125.001194,192.168.15.3,192.168.15.4,MQTT,70,NaN,104.0,1883,60501,Publish Ack (id=104)
1058,3182,125.017071,192.168.15.4,192.168.15.3,MQTT,97,Deviceb'Device - 1',104.0,36061,1883,Publish Message (id=104) [Deviceb'Device - 1']


In [3]:
devices = {}
#  : devices será um dicionario que irá ter uma chave referente ao device e o valor será um dataframe referente aos respectivos devices 
for port in ports:  
  array_publish = data.loc[data["Source Port"] == port].dropna(subset=['Message Identifier'])
  array_publish_ack = data.loc[data["Destination Port"] == port].dropna(subset=['Message Identifier'])
  device = {"publish":array_publish, "publish_ack" : array_publish_ack}
  devices[port] = device

In [4]:
data = {}
for port in ports:
    RTT = pd.DataFrame(columns=["Time_publish", "Time_ack"])
    for index, row in devices[port]["publish"].iterrows():
        id = int(row["Message Identifier"])
        time_publish = row["Time"]
        RTT.loc[id, "Time_publish"] = time_publish 

    for index, row in devices[port]["publish_ack"].iterrows():
        id = int(row["Message Identifier"])
        time_publish = row["Time"]
        RTT.loc[id, "Time_ack"] = time_publish
    RTT.loc[id, "Time_ack"] = time_publish
    RTT["RTT"]  =(RTT["Time_ack"] - RTT["Time_publish"] )* 1000
    data[port] = RTT


## Medidas de Tendência Central Por Device

In [5]:
i=1
data_mean = []
data_median = []
data_std = []
data_min = []
data_max = []

for port in ports:
  data_mean.append(data[port]["RTT"].mean())
  data_median.append(data[port]["RTT"].mean())
  data_std.append(data[port]["RTT"].mean())
  data_min.append(data[port]["RTT"].mean())
  data_max.append(data[port]["RTT"].mean())

  print("(Device-{})Mean Round Trip Time = {}".format(i ,data[port]["RTT"].mean()))
  print("(Device-{})Median Round Trip Time = {}".format(i ,data[port]["RTT"].median()))
  print("(Device-{})Standard deviation  Round Trip Time = {}".format(i ,data[port]["RTT"].std()))
  print("(Device-{})Min Round Trip Time = {}".format(i ,data[port]["RTT"].min()))
  print("(Device-{})Max Round Trip Time = {}".format(i ,data[port]["RTT"].max()))
  print("---------------------------------------------------------------")

  i+=1

(Device-1)Mean Round Trip Time = 4.601038692308648
(Device-1)Median Round Trip Time = 3.602002999999243
(Device-1)Standard deviation  Round Trip Time = 3.355256425549816
(Device-1)Min Round Trip Time = 2.1681919999991806
(Device-1)Max Round Trip Time = 24.478406000000064
---------------------------------------------------------------
(Device-2)Mean Round Trip Time = 4.39981813461584
(Device-2)Median Round Trip Time = 3.793070000000398
(Device-2)Standard deviation  Round Trip Time = 2.6520343346207205
(Device-2)Min Round Trip Time = 1.6460800000004383
(Device-2)Max Round Trip Time = 17.82986799999975
---------------------------------------------------------------
(Device-3)Mean Round Trip Time = 3.8112378173072403
(Device-3)Median Round Trip Time = 2.815661500001454
(Device-3)Standard deviation  Round Trip Time = 3.4018317510954783
(Device-3)Min Round Trip Time = 1.7406700000002218
(Device-3)Max Round Trip Time = 23.91278400000374
--------------------------------------------------------

## Medidas de Tendência Central Geral

In [6]:
from statistics import mean 

print("Mean Round Trip Time = {}".format(mean(data_mean)))
print("Median Round Trip Time = {}".format(mean(data_median)))
print("Standard deviation  Round Trip Time = {}".format(mean(data_std)))
print("Min Round Trip Time = {}".format(mean(data_min)))
print("Max Round Trip Time = {}".format(mean(data_max)))


Mean Round Trip Time = 4.0392205326926645
Median Round Trip Time = 4.0392205326926645
Standard deviation  Round Trip Time = 4.0392205326926645
Min Round Trip Time = 4.0392205326926645
Max Round Trip Time = 4.0392205326926645
